# IEMS 395-1 HW1(template for your own solution)

# Newton's method with line search

Copyright 2019, Andreas Waechter

Distribution of this file is not allowed without explicit permission of the author.

In [ ]:
# Make NumPy module available for numerical computations
import numpy as np

### Initialize options for Newton's algorithm

In [ ]:
def minimizeObjInitOptions():
    '''
    Option initialization for minimize_obj function

    Initialize algorithm options with default values

    Return values:
        options:
            This is a dictionary with fields that correspond to algorithmic
            options of our method.  In particular:

            max_iter:
                Maximum number of iterations
            tol:
                Convergence tolerance
            step_type:
                Different ways to calculate the search direction:
               'Steepest Descent'
               'Newton'
            init_alpha:
                first trial step size to attempt during line search
            suff_decrease_c1:
                coefficient in sufficient decrease condition
            output_level:
                Amount of output printed
                0: No output
                1: Only summary information
                2: One line per iteration (good for debugging)
    '''
    # we first need to create an empty dictionary before we can add entries
    options = {}

    # Now set a default value for each of the options
    options['max_iter'] = 1e6
    options['tol'] = 1e-6
    options['step_type'] = 'Newton'
    options['init_alpha'] = 1.
    options['suff_decrease_c1'] = 1e-4
    options['output_level'] = 2

    return options

### Rosenbrock objective function

In [ ]:
class Rosenbrock:
    '''
    Implementation of the Rosenbrock objective function

    a*(x[1] - x[0]**2)**2 + (b - x[0])**2

    Parameters:
        a: scalar, see formula
        b: scalar, see formula
    '''

    def __init__(self, a=100.0, b=1.0):
        '''
        Initialize object with parameters a and b in the formula.
        '''
        self._a = a
        self._b = b

    def value(self, x):
        '''
        Compute value of objective function at point x
        '''
        a = self._a
        b = self._b
        val = a*(x[1] - x[0]**2)**2 + (b - x[0])**2
        return val

    def gradient(self, x):
        '''
        Compute gradient of objective function at point x
        '''
        a = self._a
        b = self._b
        grad = np.array([0, 0])
        return grad

    def hessian(self, x):
        '''
        Compute Hessian of objective function at point x
        '''
        a = self._a
        Hess = 0
        return Hess

### Newton's algorithm

In [ ]:
def minimizeObjective(x_start, objFunc, options):
    '''
    Optimization method for unconstrained optimization

    This is the template for your solution.  You need to add or change
    things, not only at the places pointed out explicitly

    Input arguments:
        x_start:
            Starting point
        objFunc:
            Objective function object.  It must have the following methods,
            where x is the point where the quantity should be evaluated.

            val = value(x)
                returns value of objective function at x
            grad = gradient(x)
                returns gradient of objective function at x
            Hess = hessian(x)
                return Hessian of objective function at x
        options:
            This is a dictionary with options for the algorithm.
            For details see the minimizeObjInitOptions function.

    Return values:
        status:
           Return code indicating reason for termination:
            0:  Optimal solution found (convergence tolerance satisfied)
           -1:  Maximum number of iterations exceeded
           -2:  Search direction is not a descent direction
          -99:  Unknown error (bug?)
        x_sol:
            Approximate critical point (or last iterate if there is a failure)
        f_sol:
            objective function value at x_sol
        stats:
            Dictionary with statistics for the run.  Its fields are
            norm_grad      Norm of gradient at final iterate
            num_iter       Number of iterations taken
            num_func_evals Number of function evaluations needed
    '''

    # initialize iteration counter
    iter_count = 0

    # initialize return flag (set to -99 so that we do not accidentally
    # declare success.)
    status = -99
    
    # get parameters out put the options dictionary
    max_iter = options['max_iter']
    tol = options['tol']
    step_type = options['step_type']
    init_alpha = options['init_alpha']
    output_level = options['output_level']

    # initialize current iterate
    #
    # ****Important note:****
    # we need to make a copy of the x_start array.  If we would
    # just write "x_k = x_start", then the Python variables x_k and x_start
    # would be different names for the same array, and then changing
    # x_k would also change x_start.  Since we do not want to change what is
    # stored as the starting point, we need to make a copy here.
    x_k = np.copy(x_start)

    # initialize current function value, gradient, and infinity norm of the
    # gradient
    f_k = objFunc.value(x_k)
    grad_k = objFunc.gradient(x_k)
    norm_grad_k = np.linalg.norm(grad_k, np.inf)

    # initialize counter for function evaluations
    num_func_evals = 1

    # determine how many variables are in the problem
    num_vars = len(x_start)

    # initialize search direction and its length to zero
    p_k = np.zeros(num_vars)
    norm_pk = 0.0

    # initialize step size to zero (this is only necessary so that we can print
    # an output line before a meaningful step size has actually been computed.)
    alpha_k = 0.0

    # initialize the counter for the function evaluations needed in a
    # particular iteration
    num_func_it = 0

    # Print header and zero-th iteration for output
    if output_level >= 2:
        # (This is just a fancy way to create a string.  The '%s' formatting
        # makes it easy to align the header with the actual output.)
        output_header = '%6s %23s %9s %9s %6s %9s' % \
            ('iter', 'f', '||p_k||', 'alpha', '#func', '||grad_f||')
        print(output_header)
        print('%6i %23.16e %9.2e %9.2e %6i %9.2e' %
              (iter_count, f_k, norm_pk, alpha_k, num_func_it, norm_grad_k))

    ###########################
    # Beginning of main Loop
    ###########################

    # We continue the main loop until the termination tolerance is met.
    while 1:

        ##############################################################
        # Check termination tests
        ##############################################################
        if 1. <= tol:
            # Termination tolerance met
            status = 0
            break

        if iter_count >= 100:
            # Set flag to indicate the maximum number of iterations has been
            # exceeded
            status = -1
            # The following command says that we now want to leave the current
            # loop (which is the while loop here).  The program execution will
            # resume immediately after the end of the loop
            break

        ##############################################################
        # Compute search direction
        ##############################################################
        if step_type == 'Newton':
            p_k = np.zeros(num_vars)  # Obviously, put something reasonable here
        elif step_type == 'gradient_descent':
            p_k = np.zeros(num_vars)  # Obviously, put something reasonable here
        else:
            raise ValueError('Invalid value for options[step_type]')

        ##############################################################
        # Perform the backtracking Armijo line search
        ##############################################################

        # initialize step size
        alpha_k = init_alpha

        # Compute trial point and objective value at trial point
        x_trial = x_k + alpha_k*p_k
        f_trial = objFunc.value(x_trial)
        num_func_it = 1
        
        # Put your code here

        # Update iterate
        x_k = np.copy(x_trial)
        f_k = f_trial

        # Compute gradient and its norm at the new iterate
        grad_k = objFunc.gradient(x_k)
        norm_grad_k = np.linalg.norm(grad_k, np.inf)

        # For the output, compute the norm of the step
        norm_pk = np.linalg.norm(p_k, np.inf)

        # Update counter for total number of function evaluations
        num_func_evals += num_func_it

        # Increase the iteration counter
        iter_count += 1

        # Iteration output
        if output_level >= 2:
            # Print the output header every 10 iterations
            if iter_count % 10 == 0:
                print(output_header)
            print('%6i %23.16e %9.2e %9.2e %6i %9.2e' %
                  (iter_count, f_k, norm_pk, alpha_k, num_func_it, norm_grad_k))

    ###########################
    # End of main loop
    ###########################

    ###########################
    # Finalize results
    ###########################

    # Set last iterate as the one that is returned, together with its objective
    # value
    x_sol = x_k
    f_sol = f_k

    # Set the statistics
    stats = {}
    stats['num_iter'] = iter_count
    stats['norm_grad'] = norm_grad_k
    stats['num_func_evals'] = num_func_evals

    # Final output message
    if output_level >= 1:
        print('')
        print('Final objective.................: %g' % f_sol)
        print('||grad|| at final point.........: %g' % norm_grad_k)
        print('Number of iterations............: %d' % iter_count)
        print('Number of function evaluations..: %d' % num_func_evals)
        print('')
        if status == 0:
            print('Exit: Critical point found.')
        elif status == -1:
            print('Exit: Maximum number of iterations (%d) exceeded.' %
                  iter_count)
        elif status == -2:
            print('Exit: Search direction is not a descent direction.')
        elif status == -3:
            print('Exit: Exit: Step size becomes zero.')
        else:
            print('ERROR: Unknown status value: %d\n' % status)

    # Return output arguments
    return status, x_sol, f_sol, stats


### Solve Rosenbrock with steepest descent from first starting point

In [ ]:
options = minimizeObjInitOptions()
options['step_type'] = 'gradient_descent'
x_start = np.array([1.2, 1.2])
objFunc = Rosenbrock()

status, x_sol, f_sol, stats = minimizeObjective( x_start, objFunc, options )
